# Meta Kaggle: Count User Activities

This notebook simply counts appearances of UserId in the various Meta Kaggle data tables.


In [1]:
import gc, os, sys, time
import pandas as pd, numpy as np
from itertools import combinations
from IPython.display import HTML, display

pd.options.display.max_rows = 200

In [2]:
IN_DIR = '../input'
len(os.listdir(IN_DIR))

In [3]:
users = pd.read_csv(f'{IN_DIR}/Users.csv', index_col='Id')
users.shape

In [4]:
EXCLUDE_USERS = [2080166] # Kaggle Kerneler - very high stats that distort the league tables!

users.loc[EXCLUDE_USERS].T

In [5]:
users = users.drop(EXCLUDE_USERS)
users.shape

In [6]:
users.head()

In [7]:
def columns(fn):
    df = pd.read_csv(fn, nrows=5)
    return df.columns

def user_columns(fn):
    return [c for c in columns(fn) if 'UserId' in c]

For each csv

 -        read user id columns
 -        add counts to main users df  - format is Count_Table_Column, e.g. Count_Kernels_AuthorUserId is how many Kernels they have authored.

In [8]:
for f in os.listdir(IN_DIR):
    if '.csv' in f:
        csv = IN_DIR+'/'+f
        cols = user_columns(csv)
        if len(cols) < 1:
            continue
        table = f.replace('.csv', '')
        df = pd.read_csv(csv, usecols=cols)
        for col in cols:
            tag = f'Count_{table}_{col}'
            print(tag)
            vc = df[col].value_counts()
            ser = users.index.map(vc)
            users[tag] = ser.fillna(0).astype('int32')

Here are some rough meanings:

| Count Column | Meaning |
|---|---|
| Datasets_CreatorUserId | create a dataset |
| Datasets_OwnerUserId | own a dataset |
| DatasetVersions_CreatorUserId | create a dataset version |
| DatasetVotes_UserId | vote for a dataset |
| Datasources_CreatorUserId | create a datasource |
| ForumMessages_PostUserId | post a forum message |
| ForumMessageVotes_FromUserId | vote for a forum message |
| ForumMessageVotes_ToUserId | receive a forum (up)vote |
| Kernels_AuthorUserId | author a kernel |
| KernelVersions_AuthorUserId | author a kernelversion |
| KernelVotes_UserId | vote for a kernel |
| Submissions_SubmittedUserId | submit to a competition |
| TeamMemberships_UserId | enter a competition (agree to rules) |
| UserAchievements_UserId | reach a new achievement milestone |
| UserFollowers_UserId | follow a user |
| UserFollowers_FollowingUserId | get followed by a user |
| UserOrganizations_UserId | add an organization |


# League Tables

Show users with highest counts for each columns.

In [9]:
N_SHOW = 50

In [10]:
def user_name_link(r):
    return f'<a href="https://www.kaggle.com/{r.UserName}">{r.DisplayName}</a>'

TIERS = np.asarray([
    '<font color=green>novice</font>',
    '<font color=blue>novice</font>',
    '<font color=purple>expert</font>',
    '<font color=orange>master</font>',
    '<font color=gold>grandmaster</font>',
    '<font color=black>staff</font>',
])

def league_table(col, src_df=users):
    display(HTML("<H1>" + col.replace('_', ' ') + "</H1>"))
    df = src_df.sort_values(col, ascending=False).head(N_SHOW)
    uid = df.apply(user_name_link, axis=1)
    df.pop('UserName')
    df.pop('DisplayName')
    df.insert(0, 'Tier', TIERS[df.PerformanceTier])
    df.insert(0, 'DisplayName', uid)
    df['Rank'] = df[col].rank(method='min', ascending=False).astype(int)
    return df[['Rank','DisplayName','Tier',col]].style.bar(subset=[col], vmin=0)

# for c in activity_sums.index: print(f'league_table("{c}")')

In [11]:
league_table("Count_TeamMemberships_UserId")

In [12]:
league_table("Count_Submissions_SubmittedUserId")

In [13]:
league_table("Count_KernelVotes_UserId")

In [14]:
league_table("Count_DatasetVotes_UserId")

In [15]:
league_table("Count_KernelVersions_AuthorUserId")

In [16]:
league_table("Count_Kernels_AuthorUserId")

In [17]:
league_table("Count_ForumMessages_PostUserId")

In [18]:
league_table("Count_UserFollowers_UserId")

In [19]:
league_table("Count_ForumMessageVotes_FromUserId")

In [20]:
league_table("Count_UserFollowers_FollowingUserId")

In [21]:
league_table("Count_ForumMessageVotes_ToUserId")

In [22]:
league_table("Count_DatasetVersions_CreatorUserId")

In [23]:
league_table("Count_Datasets_CreatorUserId")

In [24]:
league_table("Count_Datasources_CreatorUserId")

In [25]:
league_table("Count_Datasets_OwnerUserId")

In [26]:
league_table("Count_UserOrganizations_UserId")

In [27]:
all_col_counts = users.columns[users.columns.str.startswith('Count_')]
len(all_col_counts)

Count_UserAchievements_UserId - nearly all users have 3 achievements by default, so ignore that

In [28]:
users.Count_UserAchievements_UserId.value_counts()

In [29]:
count_cols = [c for c in all_col_counts if c != 'Count_UserAchievements_UserId']
len(count_cols)

Simply SUM up all activities on Kaggle, way over two million have yet to *do* anything!

In [30]:
users[count_cols].sum(1).value_counts().head()

Fork & put your username here to see the sum of all your activities on Kaggle.

In [31]:
users.query('UserName=="jtrotman"').T

Sum the columns as boolean to count the *variety* of things each user has done.

In [32]:
users['Sum_Activity_Flags'] = (users[count_cols]>0).sum(1)

Again we see over 2 million have not done anything, whilst over 60 have done them **ALL**!

In [33]:
users.Sum_Activity_Flags.value_counts()

What proportion have not done anything?

In [34]:
(users.Sum_Activity_Flags==0).mean()

There are 16 activities at most

In [35]:
users.Sum_Activity_Flags.max()

# Über Kagglers

How many have done them all? These are the *über Kagglers*...

In [36]:
idx = users.Sum_Activity_Flags==users.Sum_Activity_Flags.max()
idx.sum()

In [37]:
show = ['UserName', 'DisplayName', 'RegisterDate', 'PerformanceTier']

In [38]:
users[idx][show]

# Über Kagglers 2

Another way: who has interacted with the site the most?

In [39]:
league_table('Total_Activities', users.assign(Total_Activities=users[count_cols].sum(1)))

# Counting Separate Activities

Utility function to count users based on their activities.

In [40]:
def users_with_n_activities(n, min_count=0):
    bi_sum = users.Sum_Activity_Flags==n
    for cols in combinations(count_cols, n):
        idx = bi_sum
        for c in cols:
            idx = (idx & (users[c]>0))
            n = idx.sum()
            if n<min_count:
                break
        if n>=min_count:
            yield (n,) + cols

def users_with_n_activities_df(n, min_count=0):
    df = pd.DataFrame.from_records(
        users_with_n_activities(n, min_count),
        columns=['Count'] + list(range(n))
    )
    return df

Of users with 1 activity, the largest group is over 400k users: they have accepted rules for a competition, but not submitted, or posted messages, or voted etc...

Next is Count_DatasetVotes_UserId: they have only voted for a dataset. (Hmmm. Who are they? Any sockpuppets in there? Do they have long streaks of consecutive user Ids?)

Then Count_KernelVotes_UserId: they have only voted for a kernel. (Hmmm.)

Currently 4 have managed to submit to a competition without being in a team(?)

In [41]:
users_with_n_activities_df(1).sort_values('Count', ascending=False).reset_index(drop=True)

Of users with 2 activities, largest group is currently Count_Submissions_SubmittedUserId and Count_TeamMemberships_UserId, those who've entered a competition AND submitted...

In [42]:
users_with_n_activities_df(2).sort_values('Count', ascending=False).reset_index(drop=True)

Users with 3 activities...

In [43]:
users_with_n_activities_df(3, min_count=2000).sort_values('Count', ascending=False).reset_index(drop=True)

# Specialised Counts

Count users who are totally dormant, or have *at most* entered & submitted to competitions. (Following this thread actually leads to interesting clusters that *appear* to be sockpuppet accounts used for submission probing in past competitions - accounts that remain on completed competition leaderboards...)

In [44]:
users.shape

In [45]:
entered = users.Count_TeamMemberships_UserId>0
submitted = users.Count_Submissions_SubmittedUserId>0

idx = (
 (users.Sum_Activity_Flags==0)
 | 
 ((users.Sum_Activity_Flags==1) & (entered))
 | 
 ((users.Sum_Activity_Flags==2) & (entered) & (submitted))
)

Quite a lot...

In [46]:
idx.sum()

In [47]:
idx.mean()

How many have done more than enter competitions?

In [48]:
users.shape[0] - idx.sum()

# Columnwise Counts

Sum up which activites are most popular.

Over 700k enter/sign up to one or more competitions but of those only 200k or so get around to making a submission.

In [49]:
activity_sums = (users[count_cols]>0).sum(0)
activity_sums = activity_sums.sort_values(ascending=False)
activity_sums

# Tier Counts

Show counts of tiers.

In [50]:
performanceTiers = np.asarray(['novice', 'contributor', 'expert', 'master', 'gm', 'staff'])

In [51]:
users.PerformanceTier.value_counts()

In [52]:
vc = users.PerformanceTier.value_counts()

In [53]:
vc.index = performanceTiers[vc.index]

In [54]:
vc